In [113]:
#kense's DBpedia queries
from SPARQLWrapper import SPARQLWrapper
from bs4 import BeautifulSoup
import requests

def query_DBpedia (query):
    '''
    obtain the JSON of the query to the DBpedia database.
    '''

    db = SPARQLWrapper("http://dbpedia.org/sparql/") #query DBpedia via SPARQL
    db.setQuery(query)
    db.method = "POST"
    db.setReturnFormat('json')
    db.queryType = "SELECT"
    result = db.query().convert()

    return result

def inquire_place (name):
    
    #query DBpedia based on name
    query = """PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX geo:  <http://www.w3.org/2003/01/geo/wgs84_pos#>
    SELECT ?place ?label ?lat ?lng
    WHERE { ?place a dbo:Place .
    ?place foaf:name ?label . 
    FILTER (?label IN ( '%s'@en)) .
    ?place geo:lat ?lat .
    ?place geo:long ?lng .}
    """ % name

    inquiry = query_DBpedia(query)
    
    #no results
    if len(inquiry['results']['bindings']) == 0:
        return None
    
    else:
        #query results
        dbpedia_url = inquiry['results']['bindings'][0]['place']['value']
        lat = inquiry['results']['bindings'][0]['lat']['value']
        lng = inquiry['results']['bindings'][0]['lng']['value']
        name = inquiry['results']['bindings'][0]['label']['value']
        comment = 'none' #defaults none 
        country = 'none'
        postalCode = 'none'
        max_ele, min_ele, ele = 'none', 'none', 'none'
        
        #download XML file
        xml_file = requests.get(dbpedia_url)
        open('./%s.xml'%name, 'wb').write(xml_file.content)
        
        #get comment, and some relevant info
        fp = open("./%s.xml"%name, "r")
        soup = BeautifulSoup(fp, 'xml')
        
        #check spans
        for item in soup.find_all("span"):
            if item.get('property') == 'dbo:abstract': #get comment
                if item.get('xml:lang') == "en":
                    comment = item.get_text()
            if item.get('property') == 'dbo:postalCode': #get postalCode
                postalCode = item.get_text()
            if item.get('property') == "dbo:maximumElevation": #max elevation
                max_ele = item.get_text()
            if item.get('property') == "dbo:minimumElevation": #min elevation
                min_ele = item.get_text()
            if not max_ele == 'none' and not min_ele == 'none': #elevation calc
                ele = str((float(max_ele)+float(min_ele))/2)
        
        #check a's
        for item in soup.find_all("a"):
            if item.get('rel') == "dbo:country": #get country
                country = item.get_text()
        
        #output
        print('DBpedia entry found!')
        print('Downloaded XML file...')
        return {"name": name, "lat": lat, "lng": lng, "url": dbpedia_url, "elevation": ele, "country": country, "zip": postalCode, "info": comment}

In [114]:
#using Refuge Maison TTL file as a reference
def get_all_dbp():
    
    #we uploaded all ttl files into GRAPHDB
    graphdb = "http://10.20.241.27:7200/repositories/WEB2019"
    
    #we just want the names so we can query dbpedia via name
    query = '''
    PREFIX :<http://www.topografix.com/GPX/1/1>
    SELECT ?wpt ?name
    WHERE {{?wpt :hasname ?name .}}
    '''
    
    db = SPARQLWrapper(graphdb)
    db.setQuery(query)
    db.method = "POST"
    db.setReturnFormat('json')
    db.queryType = "SELECT"
    result = db.query()
    fp = result.convert()
    
    names_list = []
    for i in range(0, len(fp['results']['bindings'])):
        target_name = fp['results']['bindings'][i]['name']
        if target_name['type'] == 'literal':
            target_name = target_name['value']
            target_name = target_name.replace("'", "")
            target_name = target_name.replace(".", "")
            names_list.append(target_name)
            
    
    dbp_entries = []
    for item in names_list:
        if inquire_place(item) != None:
            dbp_entries.append(inquire_place(item))
    
    return dbp_entries

In [115]:
places = get_all_dbp()

DBpedia entry found!
Downloaded XML file...
DBpedia entry found!
Downloaded XML file...
DBpedia entry found!
Downloaded XML file...
DBpedia entry found!
Downloaded XML file...
DBpedia entry found!
Downloaded XML file...
DBpedia entry found!
Downloaded XML file...
DBpedia entry found!
Downloaded XML file...
DBpedia entry found!
Downloaded XML file...


In [116]:
places

[{'name': 'Courmayeur',
  'lat': '45.7833',
  'lng': '6.96667',
  'url': 'http://dbpedia.org/resource/Courmayeur',
  'elevation': '3017.0',
  'country': 'dbr:Italy',
  'zip': '11013',
  'info': 'Courmayeur is a town and comune in northern Italy, in the autonomous region of Aosta Valley. At an elevation of 1,224 m (4,016 ft) above sea level, it is located at the foot of the southern side of Mont Blanc, at 4,810 m (15,781 ft) the highest point in the Alps and western Europe (see Seven Summits), and is crossed by the Dora Baltea. Courmayeur shares administration of Mont Blanc with its neighboring commune of Saint-Gervais-les-Bains in France, and is consequently able to claim the title of highest commune in Italy. Courmayeur also shares access to the famous glacial ski run of the Vallée Blanche with another French town, Chamonix, which sits at the opposite, northern, side of the Mont Blanc massif.'},
 {'name': 'Les Granges',
  'lat': '48.0589',
  'lng': '4.0542',
  'url': 'http://dbpedia.o